<a href="https://colab.research.google.com/github/Eunhye1109/Practical-Project/blob/EH/250723_AI%EC%9A%94%EC%95%BD_KoBART_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ✅ 0. 필수 패키지 설치
!pip install -q sentence-transformers
!pip install -q transformers
!pip install -q PyMuPDF

# ✅ 1. 임포트
import os
import re
import fitz  # PyMuPDF
import torch
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 74.2 MB/s eta 0:00:00


In [45]:
# ✅ 필요 라이브러리 설치 (처음 실행 시)
!pip install sentence-transformers transformers pdfplumber --quiet
!pip install -q sentence-transformers transformers

!pip install sentence-transformers transformers --quiet


In [59]:
# ✅ 0. 필수 패키지 설치
!pip install -q sentence-transformers transformers PyMuPDF

# ✅ 1. 임포트
import os
import re
import fitz  # PyMuPDF
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer, util

In [60]:
# ✅ 2. KoBART 요약 모델 로딩
tokenizer_kobart = AutoTokenizer.from_pretrained("digit82/kobart-summarization")
model_kobart = AutoModelForSeq2SeqLM.from_pretrained("digit82/kobart-summarization")

def summarize_kobart(text, max_length=60, min_length=20):
    inputs = tokenizer_kobart([text], max_length=512, truncation=True, return_tensors="pt")
    summary_ids = model_kobart.generate(
        inputs["input_ids"],
        max_length=max_length,
        min_length=min_length,
        num_beams=4,
        early_stopping=True
    )
    return tokenizer_kobart.decode(summary_ids[0], skip_special_tokens=True)

# ✅ 3. 기업명 입력
company = input("요약할 기업명을 입력하세요: ").strip()

# ✅ 4. 경로 설정 및 PDF 검색
pdf_dir = "/content/drive/MyDrive/2. 취업 부업/7. SW개발자과정/3. 실전프로젝트/[실전] 재무 관련 데이터_호성 팀/PDF 사업 보고서"

if not os.path.exists(pdf_dir):
    print("❌ 경로가 존재하지 않습니다.")
else:
    files = os.listdir(pdf_dir)
    matched = [f for f in files if company in f and f.endswith(".pdf")]

    if not matched:
        print(f"❌ '{company}' 관련 PDF 파일을 찾을 수 없습니다.")
    else:
        pdf_file = os.path.join(pdf_dir, matched[0])
        print(f"✅ 찾은 파일: {pdf_file}")

        # ✅ 5. PDF 텍스트 추출
        with fitz.open(pdf_file) as doc:
            raw_text = ""
            for page in doc:
                raw_text += page.get_text()

        # ✅ 6. 전처리 및 문장 나누기
        text = re.sub(r'\[\d+\]', '', raw_text)
        text = re.sub(r'\n+', ' ', text)
        sentences = re.split(r'(?<=[.?!])\s+', text)

        def is_valid_sentence(s):
            if len(s.strip()) < 10: return False
            digits = sum(c.isdigit() for c in s)
            if digits / max(len(s), 1) > 0.2: return False
            if len(re.findall(r'[가-힣a-zA-Z]', s)) < 5: return False
            return True

        filtered_sentences = [s.strip() for s in sentences if is_valid_sentence(s)]
        print(f"📚 정제된 문장 수: {len(filtered_sentences)}")

        # ✅ 7. SBERT 임베딩 및 주제 정의
        model_sbert = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
        sentence_embeddings = model_sbert.encode(filtered_sentences, convert_to_tensor=True)

        topics = {
            "📊 재무 요약": "재무 성과 및 수익에 대한 요약",
            "🛡️ 리스크 및 부채": "부채 및 재무 리스크",
            "🚀 사업 전략 및 성장": "성장 전략과 비즈니스 계획",
            "🔬 기술 및 R&D": "기술 투자 및 연구개발",
            "🌐 글로벌 진출": "해외 시장 및 글로벌 전략",
            "🏢 조직 및 인력": "조직 구성 및 인력 운영"
        }
        topic_embeddings = model_sbert.encode(list(topics.values()), convert_to_tensor=True)

        # ✅ 8. 토픽별 요약 수행
        summary_result = {}
        for i, topic in enumerate(topics.keys()):
            cosine_scores = util.pytorch_cos_sim(topic_embeddings[i], sentence_embeddings)[0]
            top_k = torch.topk(cosine_scores, k=min(5, len(filtered_sentences)))
            top_sentences = [filtered_sentences[idx] for idx in top_k.indices]
            joined_text = " ".join(top_sentences)
            summary = summarize_kobart(joined_text)
            summary_result[topic] = summary.strip()

        # ✅ 9. 결과 출력 및 저장
        final_output = ""
        for sec, s in summary_result.items():
            final_output += f"\n\n## {sec}\n- {s.strip()}\n"

        output_path = pdf_file.replace(".pdf", "_요약결과.txt")
        with open(output_path, "w", encoding="utf-8") as f:
            f.write(final_output.strip())

        print("✅ 요약 완료! 저장 위치:", output_path)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.


요약할 기업명을 입력하세요: 일동제약
❌ '일동제약' 관련 PDF 파일을 찾을 수 없습니다.


In [61]:
import unicodedata

# 회사명 정리
company = input("요약할 기업명을 입력하세요: ").strip()
company = unicodedata.normalize("NFC", company)

# 경로 설정
pdf_dir = "/content/drive/MyDrive/2. 취업 부업/7. SW개발자과정/3. 실전프로젝트/[실전] 재무 관련 데이터_호성 팀/PDF 사업 보고서"

if not os.path.exists(pdf_dir):
    print("❌ 경로가 존재하지 않습니다.")
else:
    files = os.listdir(pdf_dir)

    # 파일명 정규화 후 비교
    matched = [f for f in files if company in unicodedata.normalize("NFC", f) and f.endswith(".pdf")]

    if not matched:
        print(f"❌ '{company}' 관련 PDF 파일을 찾을 수 없습니다.")
    else:
        print(f"✅ 찾은 파일: {matched[0]}")


요약할 기업명을 입력하세요: 일동제약
✅ 찾은 파일: 일동제약 사업보고서 (2024.12).pdf


## 최종 우리말 요약 기능

In [67]:
# ✅ 0. 패키지 설치 (코랩에서 필요 시 실행)
!pip install -q sentence-transformers transformers PyMuPDF

# ✅ 1. 임포트
import os, re, unicodedata
import fitz  # PyMuPDF
import torch
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# ✅ 2. KoBART 요약 모델 로딩
tokenizer_kobart = AutoTokenizer.from_pretrained("digit82/kobart-summarization", use_fast=False)
model_kobart = AutoModelForSeq2SeqLM.from_pretrained("digit82/kobart-summarization")

def summarize_kobart(text, max_length=120, min_length=30):
    inputs = tokenizer_kobart([text], max_length=512, truncation=True, return_tensors="pt")
    summary_ids = model_kobart.generate(
        inputs["input_ids"],
        max_length=max_length,
        min_length=min_length,
        num_beams=4,
        early_stopping=True
    )
    return tokenizer_kobart.decode(summary_ids[0], skip_special_tokens=True)

# ✅ 3. SBERT 임베딩 모델
model_sbert = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# ✅ 4. 사용자 입력
company = input("요약할 기업명을 입력하세요: ").strip()
company = unicodedata.normalize("NFC", company)

# ✅ 5. PDF 파일 읽기 (정확한 경로로 수정)
pdf_dir = "/content/drive/MyDrive/2. 취업 부업/7. SW개발자과정/3. 실전프로젝트/[실전] 재무 관련 데이터_호성 팀/PDF 사업 보고서"
assert os.path.exists(pdf_dir), "경로가 존재하지 않습니다."


files = os.listdir(pdf_dir)
matched = [f for f in files
           if company in unicodedata.normalize("NFC", f) and f.lower().endswith(".pdf")]

if not matched:
    raise FileNotFoundError(f"❌ '{company}' 관련 PDF 파일이 없습니다.")
pdf_path = os.path.join(pdf_dir, matched[0])
print("✅ 찾은 PDF:", pdf_path)

# ✅ 6. PDF → 텍스트 추출
doc = fitz.open(pdf_path)
raw_text = ""
for page in doc:
    raw_text += page.get_text()
doc.close()

# ✅ 7. 전처리: 문장 분리 및 필터링
text = re.sub(r'\[\d+\]', '', raw_text)
text = re.sub(r'\s+', ' ', text)
sentences = re.split(r'(?<=[.?!])\s+', text)

def is_valid(s):
    s = s.strip()
    if len(s) < 10: return False
    if sum(c.isdigit() for c in s) / max(len(s),1) > 0.2: return False
    if len(re.findall(r'[가-힣a-zA-Z]', s)) < 5: return False
    return True

filtered = [s for s in sentences if is_valid(s)]
print("📚 유효 문장 수:", len(filtered))

# ✅ 8. 임베딩 계산
sent_embed = model_sbert.encode(filtered, convert_to_tensor=True)

topics = {
    # "📊 재무 요약": "재무 성과 및 수익",
    # "🛡️ 리스크 및 부채": "부채 및 리스크",
    # "🚀 사업 전략": "사업 전략 및 성장 계획",
    # "🔬 기술 및 R&D": "기술 투자 및 연구개발",
    # "🌐 글로벌 진출": "글로벌 진출 및 해외 시장",
    "📊 재무 요약": "최근 3개년 손익계산서를 기준으로 매출액, 영업이익, 당기순이익 등의 주요 재무성과를 종합적으로 요약한 내용입니다.",
    "🛡️ 리스크 및 부채": "기업의 주요 부채 현황, 유동성 및 지급능력 리스크, 채무불이행 가능성, 지급보증 및 담보제공 등에 관한 내용입니다.",
    "🚀 사업 전략": "신규 사업 추진, 투자계획, 매출 증대를 위한 전략, 사업 다각화 방안 및 향후 중장기 성장계획에 관한 설명입니다.",
    "🔬 기술 및 R&D": "연구개발 투자 현황, 주요 신약 및 기술 파이프라인 진행상황, 기술 제휴 및 연구성과에 대한 요약입니다.",
    "🌐 글로벌 진출": "해외시장 매출 비중, 글로벌 파트너십 체결, 수출 확대, 현지법인 설립 등 글로벌 진출 전략 및 성과에 관한 내용입니다.",

}

topic_embed = model_sbert.encode(list(topics.values()), convert_to_tensor=True)

# ✅ 9. 주제별 문장 추출 & 요약
result = {}
for i, title in enumerate(topics.keys()):
    sims = util.pytorch_cos_sim(topic_embed[i], sent_embed)[0]
    topk = torch.topk(sims, k=min(5, len(filtered)))
    sel = [filtered[idx] for idx in topk.indices]
    joined = " ".join(sel)
    result[title] = summarize_kobart(joined)

# ✅ 10. 결과 출력
print("\n\n======= 요약 결과 =======")
for sec, summ in result.items():
    print(f"\n## {sec}\n{summ}")


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.


요약할 기업명을 입력하세요: 알피바이오
✅ 찾은 PDF: /content/drive/MyDrive/2. 취업 부업/7. SW개발자과정/3. 실전프로젝트/[실전] 재무 관련 데이터_호성 팀/PDF 사업 보고서/알피바이오 사업보고서 (2024.12).pdf
📚 유효 문장 수: 698


======= 요약 결과 =======

## 📊 재무 요약
출자 목적 출자회사수 총 출자금액 상장 비상장 계 기초 장부 가액 증가(감소) 기말 장부 가액 취득 (처분) 평가 손익 경영참여 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

## 🛡️ 리스크 및 부채
당기법인세 자산과 부채는 법적으로 상계할 수 있는 권 리를 회사가 보유하고 있고, 순액으로 결제할 의도가 있거나 자산을 실현하는 동시에 부채를 결제하려는 의도가 있는 경우에 상계한다.

## 🚀 사업 전략
정부의 정책기조 가 시장활성화를 위한 기조가 지속되고 있으며 앞으로 바이오의약품의 성장, 외자사와의 전략적 제휴, 개별 종목들의 연구성과나 수출계약 등이 지속적으로 부각되며 향후 경제성장을 이끌 주력산업으로 성장할 것으로 예상된다.

## 🔬 기술 및 R&D
바이오의약품의 성장, 외자사와의 전략적 제휴, 개별 종목들의 연구성과나 수출계약 등이 지속적으로 부각되며 향후 경제성장을 이끌 주력산업으로 성장할 것으로 예상된다.

## 🌐 글로벌 진출
제약시장은 앞으로 바이오의약품의 성장, 외자사와의 전략적 제휴, 개별 종목들의 연구성과나 수출계약 등이 지속적으로 부각되며 향후 경제성장을 이끌 주력산업으로 성장할 것으로 예상된다.


## 쉬운 말 요약 버전

In [ ]:
# ✅ 0. 패키지 설치 (코랩에서 필요 시 실행)
!pip install -q sentence-transformers transformers PyMuPDF

# ✅ 1. 임포트
import os, re, unicodedata
import fitz  # PyMuPDF
import torch
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# ✅ 2. KoBART 요약 모델 로딩
tokenizer_kobart = AutoTokenizer.from_pretrained("digit82/kobart-summarization", use_fast=False)
model_kobart = AutoModelForSeq2SeqLM.from_pretrained("digit82/kobart-summarization")

def summarize_kobart(text, max_length=120, min_length=30):
    inputs = tokenizer_kobart([text], max_length=512, truncation=True, return_tensors="pt")
    summary_ids = model_kobart.generate(
        inputs["input_ids"],
        max_length=max_length,
        min_length=min_length,
        num_beams=4,
        early_stopping=True
    )
    return tokenizer_kobart.decode(summary_ids[0], skip_special_tokens=True)

# ✅ 3. 쉬운 문장 후처리 함수
def simplify_korean(text):
    text = text.replace("해당", "이")
    text = text.replace("의하여", "로 인해")
    text = text.replace("하고자 한다", "하려고 한다")
    text = text.replace("있습니다", "있어요")
    text = text.replace("되었습니다", "됐어요")
    text = text.replace("하고 있습니다", "하고 있어요")
    text = text.replace("수 있습니다", "수 있어요")
    text = text.replace("수 없습니다", "할 수 없어요")
    text = text.replace("제공하고 있습니다", "제공 중이에요")
    return text

# ✅ 4. SBERT 임베딩 모델
model_sbert = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# ✅ 5. 사용자 입력
company = input("요약할 기업명을 입력하세요: ").strip()
company = unicodedata.normalize("NFC", company)

# ✅ 6. PDF 파일 경로 설정
pdf_dir = "/content/drive/MyDrive/2. 취업 부업/7. SW개발자과정/3. 실전프로젝트/[실전] 재무 관련 데이터_호성 팀/PDF 사업 보고서"
assert os.path.exists(pdf_dir), "경로가 존재하지 않습니다."

files = os.listdir(pdf_dir)
matched = [f for f in files
           if company in unicodedata.normalize("NFC", f) and f.lower().endswith(".pdf")]

if not matched:
    raise FileNotFoundError(f"❌ '{company}' 관련 PDF 파일이 없습니다.")
pdf_path = os.path.join(pdf_dir, matched[0])
print("✅ 찾은 PDF:", pdf_path)

# ✅ 7. PDF → 텍스트 추출
doc = fitz.open(pdf_path)
raw_text = ""
for page in doc:
    raw_text += page.get_text()
doc.close()

# ✅ 8. 전처리: 문장 분리 및 필터링
text = re.sub(r'\[\d+\]', '', raw_text)
text = re.sub(r'\s+', ' ', text)
sentences = re.split(r'(?<=[.?!])\s+', text)

def is_valid(s):
    s = s.strip()
    if len(s) < 10: return False
    if sum(c.isdigit() for c in s) / max(len(s),1) > 0.2: return False
    if len(re.findall(r'[가-힣a-zA-Z]', s)) < 5: return False
    return True

filtered = [s for s in sentences if is_valid(s)]
print("📚 유효 문장 수:", len(filtered))

# ✅ 9. 임베딩 계산
sent_embed = model_sbert.encode(filtered, convert_to_tensor=True)

topics = {
    "📊 재무 요약": "최근 3개년 손익계산서를 기준으로 매출액, 영업이익, 당기순이익 등의 주요 재무성과를 종합적으로 요약한 내용입니다.",
    "🛡️ 리스크 및 부채": "기업의 주요 부채 현황, 유동성 및 지급능력 리스크, 채무불이행 가능성, 지급보증 및 담보제공 등에 관한 내용입니다.",
    "🚀 사업 전략": "신규 사업 추진, 투자계획, 매출 증대를 위한 전략, 사업 다각화 방안 및 향후 중장기 성장계획에 관한 설명입니다.",
    "🔬 기술 및 R&D": "연구개발 투자 현황, 주요 신약 및 기술 파이프라인 진행상황, 기술 제휴 및 연구성과에 대한 요약입니다.",
    "🌐 글로벌 진출": "해외시장 매출 비중, 글로벌 파트너십 체결, 수출 확대, 현지법인 설립 등 글로벌 진출 전략 및 성과에 관한 내용입니다.",
}

topic_embed = model_sbert.encode(list(topics.values()), convert_to_tensor=True)

# ✅ 10. 주제별 문장 추출 & 요약 + 쉬운 말 변환
result = {}
for i, title in enumerate(topics.keys()):
    sims = util.pytorch_cos_sim(topic_embed[i], sent_embed)[0]
    topk = torch.topk(sims, k=min(5, len(filtered)))
    sel = [filtered[idx] for idx in topk.indices]
    joined = " ".join(sel)
    summarized = summarize_kobart(joined)
    simplified = simplify_korean(summarized)
    result[title] = simplified

# ✅ 11. 결과 출력
print("\n\n======= 요약 결과 (쉬운 말) =======")
for sec, summ in result.items():
    print(f"\n## {sec}")
    for line in summ.split(". "):
        print(f"- {line.strip()}.")


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.




======= 요약 결과 (쉬운 말) =======

## 📊 재무 요약
- (4) 당기말 및 전기말 현재 투자부동산의 공정가치는 다음과 같으며 (4) 당기말 및 전기말 현재 투자부동산의 공정가치는 다음과 같다..

## 🛡️ 리스크 및 부채
- 당기법인세자산과 부채는 법적으로 상계할 수 있는 권리를 회사가 보유하고 있고, 순액으로 결제할 의도가 있거나 자산을 실현하는 동시에 부채를 결제하려는 의도가 있는 경우에 상계하여 재무상태표에 순액으로 표시하고 있다..

## 🚀 사업 전략
- 배당절차 개선방안 이행 관련 향후 계획 당사는 배당 절차 개선방안에 대해 타상장리츠사 및 당사 사업현황 등을 고려하여 도입을 검토하겠다..

## 🔬 기술 및 R&D
- 배당절차 개선방안 이행 관련 향후 계획 당사는 배당 절차 개선방안에 대해 타상장리츠사 및 당사 사업현황 등을 고려하여 도입을 검토하겠다..

## 🌐 글로벌 진출
- 100.0 삼성물산(주) Vista Contracting and Investment Global Pte.는 100.0 삼성물산(주) Vista Contracting and Investment Global Pte.의 주요 매출처 및 매출액은 아래와 같으며 주요계약 및 연구개발활동, 주요계약 및 연구개발활동.............................................................................................................................................................
